<a href="https://colab.research.google.com/github/Julianfried/Diplomatura-en-IA-Instituto-Humai/blob/main/Machine%20Learning/Clase_7_Ejercicios_XGBoost%2C_LightGBM%2C_CatBoost_y_NGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/institutohumai/machine_learning/blob/main/7_XGBoost/Ejercicios/Ejercicios.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>

# Ejercicios XGBoost, LightGBM, CatBoost y NGBoost

Recuerden activar la opción de acelerador por hardaware "gpu" La cual se encuentra en "Entorno de ejecución" y dentro de "cambiar tipo de entorno de ejecución"



## Diseño de Modelo XGBoost para predicción de salarios
Utilizando la librería XGBoost:
1. Limpie el Dataset eliminando los valores nulos y conviertendo las variables categóricas mediante OneHotEncoding o similar.
2. Divida el conjunto de datos en datos de entrenamiento y prueba.
3. Implemente un modelo XGBoost para predecir si una persona gana más de 50K en función de las otras características y entrénelo con el set de entrenamiento.
4. Evalúe el rendimiento del modelo usando el conjunto de prueba y la métrica ROC_AUC.


In [2]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

In [3]:
# Leer el dataset
df = pd.read_csv("https://datasets-humai.s3.amazonaws.com/datasets/adult_train.csv")

# Limpieza de datos y Encoding
df = df.replace('?', np.nan)
df = df.dropna()
ohe = OneHotEncoder()
df_encoded = ohe.fit_transform(df)


# Data Split
X = df_encoded[:, :-1]
y = df_encoded[:, -1]

# Convertir las matrices dispersas a arreglos NumPy
X= X.toarray()
y= y.toarray().ravel()

# División de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X[:, :-1], y, test_size=0.25, random_state=42)

In [7]:
# Modelo
xgb_cl = xgb.XGBClassifier(objective= 'binary:logistic', n_estimators=10, seed=42, tree_method='gpu_hist')
xgb_cl.fit(X_train, y_train)
# Evaluación
preds= xgb_cl.predict(X_test)
auc_score_xgboost = roc_auc_score(y_test, preds)
print('El resultado de ROC-AUC es:',auc_score_xgboost)

El resultado de ROC-AUC es: 0.7336191994008351


## Empleo de Lightgbm para predicciones salariales
Implemente y entrene un modelo Lightgbm utilizando el dataset adult para predecir el salario de una persona. Evalúe el rendimiento del modelo usando el conjunto de datos de prueba y la métrica ROC_AUC.

In [8]:
import lightgbm as lgb

# Crear el dataset LightGBM
train_data = lgb.Dataset(X_train, label=y_train)

# Definir los parámetros del modelo LightGBM
params = {
    'objective':'binary',
    'metric':'auc',
    'seed':42
}

# Entrenar el modelo LightGBM en la GPU
lgbm_cl = lgb.train(params, train_data,num_boost_round=1)

# Evaluación
preds = lgbm_cl.predict(X_test)
auc_score_lightgmb = roc_auc_score(y_test, preds)
print('El resultado de ROC-AUC es:',auc_score_lightgmb)

[LightGBM] [Info] Number of positive: 5978, number of negative: 18151
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002902 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 24129, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.247752 -> initscore=-1.110640
[LightGBM] [Info] Start training from score -1.110640
El resultado de ROC-AUC es: 0.851819701898081


## Empleo de CatBoost para predicciones salariales
Implemente y entrene un modelo CatBoost utilizando el dataset adult para predecir el salario de una persona. Evalúe el rendimiento del modelo usando el conjunto de datos de prueba y la métrica ROC_AUC.


In [9]:
!pip install catboost

In [10]:
from catboost import CatBoostClassifier

# Modelo
catb_cl = CatBoostClassifier(verbose=0, n_estimators=100, task_type='GPU')
catb_cl.fit(X_train, y_train)
# Evaluación
preds = catb_cl.predict(X_test)
auc_score_catb = roc_auc_score(y_test, preds)
print('El resultado de ROC-AUC es:',auc_score_catb)

El resultado de ROC-AUC es: 0.7472210247634474


## Empleo de NGBoost para predicciones salariales
Implemente y entrene un modelo NGBoost utilizando el dataset adult para predecir el salario de una persona. Evalúe el rendimiento del modelo usando el conjunto de datos de prueba y la métrica ROC_AUC.

In [9]:
!pip install ngboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.4/409.4 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 9.6 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=6f0bf8773f8f0b3158865c85d316928c8efe88337cda57759daf504f5addde28
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma


Aclaración: El empleo del modelo NGBoost no podria correr en el colab debido a que este tiene un limite computacional. Debido a ello se reduce la dimensionalidad de los datos Ipara poder aplicar el modelo. Para ello se corre el siguiente codigo:

In [4]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Reducción de dimensionalidad con PCA
pca = PCA(n_components=10)  # Número de componentes principales deseados
X_pca = pca.fit_transform(X[:, :-1])

# División de datos en entrenamiento y prueba con datos reducidos por PCA
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Convertir las etiquetas a formato entero
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [6]:
from ngboost import NGBClassifier
from ngboost.distns import Bernoulli # Bernoulli actua como la distribución de la clase Dist para problemas de clasificación binaria

# Modelo
ngb_cl = NGBClassifier(Dist=Bernoulli, n_estimators=100)
ngb_cl.fit(X_train, y_train)

# Evaluación
preds = ngb_cl.predict(X_test)
auc_score_ngb = roc_auc_score(y_test, preds)
print('El resultado de ROC-AUC es:',auc_score_ngb)

[iter 0] loss=0.5599 val_loss=0.0000 scale=2.0000 norm=4.0000
El resultado de ROC-AUC es: 0.6848710881199027


## Comparación de Modelos
Compare el rendimiento de cada modelo y determine cuál se desempeña mejor en función de la métrica ROC_AUC.


In [13]:
print(f'XGBoost AUC Score: {auc_score_xgboost}')
print(f'LightGBM AUC Score: {auc_score_lightgmb}')
print(f'CatBoost AUC Score: {auc_score_catb}')
print(f'NGBoost AUC Score: {auc_score_ngb}')

XGBoost AUC Score: 0.7336191994008351
LightGBM AUC Score: 0.851819701898081
CatBoost AUC Score: 0.7472210247634474
NGBoost AUC Score: 0.6848710881199027
